In [1]:
# Packages 
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow_examples.models.pix2pix import pix2pix

from IPython.display import clear_output

import time

In [2]:
# Get username
username = os.getlogin()

# Locate the training and mask directories
image_dir = r"C:\Users\\"+ username + r"\iCloudDrive\Documents\Computer Science\Personal Projects\Considition (Image Recongition)\Training_dataset\Image\Images"
mask_dir = r"C:\Users\\" + username + r"\iCloudDrive\Documents\Computer Science\Personal Projects\Considition (Image Recongition)\Training_dataset\Masks\Mask\all"

In [3]:
def load_mask(file_path, size):
    
    mask_array = []
    
    # loop throught each mask directory
    for filename in os.listdir(file_path):
               
        # get mask path and read mask
        img_path = os.path.join(file_path, filename)
        image_cv = cv2.imread(img_path)
        
        # filter mask into single channel categories
        np_image = np.array(tf.image.resize(image_cv, size, method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)).astype('int32')
        np_image[:,:,0] = np.where(np_image[:,:,0] != 255, np_image[:,:,0], 510)
        np_image = np.sum(np_image/255.0, axis = 2)
        
        mask_array.append(np_image)
        
    # expand the last dimension from to 1 for model requirements
    mask_array = np.expand_dims(mask_array, 3) 
    
    return mask_array

In [4]:
# load image function
def load_images(file_path,size):
    
    image_array = []
    
    for filename in os.listdir(file_path):
        img_path = os.path.join(file_path, filename)
        image_cv = cv2.imread(img_path)/255
        np_image = cv2.resize(image_cv, size)
        image_array.append(np_image)
    
    return np.asarray(image_array)

In [5]:
size = (128,128)
image_array = load_images(image_dir, size)
mask_array = load_mask(mask_dir, size)

In [6]:
num_image = len(image_array)
num_mask = len(mask_array)

if num_image != num_mask:
    raise ImportError('Image data and mask data do not match!')
else:
     data_size = num_image   
    
print(num_image)
print(num_mask)

1670
1670


In [7]:
batch_size = 32
val_split = 0.2
buffer_size = 1000

In [8]:
# create train and test data size
train_size = int((1-val_split) * data_size)
test_size = int(data_size - train_size)
print(train_size)
print(test_size)

1336
334


In [9]:
# Flip the image randomly
@tf.function
def image_flip(input_image, input_mask):
    
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
    
    return input_image, input_mask

# Change the the colour of image randomly
@tf.function
def image_color(input_image, input_mask):

    input_image = tf.image.random_hue(input_image, 0.08)
    input_image = tf.image.random_saturation(input_image, 0.6, 1.6)
    input_image = tf.image.random_brightness(input_image, 0.05)
    input_image = tf.image.random_contrast(input_image, 0.7, 1.3)
    
    return input_image, input_mask

In [10]:
# Clip image and make sure image and mask are in the correct dtype
@tf.function
def clip_images(input_image,input_mask):
    return (tf.cast(tf.clip_by_value(input_image, 0, 1), tf.float32), tf.cast(input_mask,tf.int16))

# Function to combine all the data augmentation and processing functions
def dataset_transform(dataset, augmentations):
        
    # Add the augmentations to the dataset
    for f in augmentations:
        # Apply the augmentation, run 2 jobs in parallel.
        dataset = dataset.map(f, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
   
    dataset = dataset.map(clip_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset

In [11]:
# Define model downsampling 
OUTPUT_CHANNELS = 4

base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [12]:
# define upsampling using pix2pix.upsample(#filters, filter_size)
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

def unet_model(output_channels):

    # This is the last 2 layer of the model
    last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='valid', activation='softmax') #64x64 -> 128x128
    
    last_last = tf.keras.layers.Cropping2D(cropping=((1, 0), (1, 0)), data_format="channels_last")

    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs

    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    x = last(x)
    x = last_last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [13]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
model (Model)                   [(None, 64, 64, 96), 1841984     input_2[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8, 8, 512)    1476608     model[1][4]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 8, 8, 1088)   0           sequential[0][0]                 
                                                                 model[1][3]                

In [15]:
[node.op.name for node in model.outputs]

['cropping2d_4/Identity']

In [16]:
# the model_dir states where the graph and checkpoint files will be saved to
estimator_model = tf.keras.estimator.model_to_estimator(keras_model = model, \
                                                        model_dir = './Considition_CKPT_1')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Using config: {'_model_dir': './Considition_CKPT_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AD94C9F0B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_m

In [17]:
# Create another input function for from slices
def input_function(features,labels=None,shuffle=False, batch_size = 128):
    
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
    # Map the train dataset with tranformation for data augmentation
    if shuffle:
        dataset = dataset_transform(dataset, [image_flip,image_color])
        
    dataset = dataset.map(lambda features, labels: ({'input_2':features}, labels))
    
    if shuffle:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

input_function(image_array, mask_array, True)

<BatchDataset shapes: ({input_2: (None, 128, 128, 3)}, (None, 128, 128, 1)), types: ({input_2: tf.float32}, tf.int16)>

In [18]:
# TRAINING 
EPOCHS = 1
STEPS = train_size // batch_size * EPOCHS

estimator_model.train(input_fn = lambda: input_function(image_array, mask_array, True), steps = STEPS)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='./Considition_CKPT_1\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ./Considition_CKPT_1\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 165 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Considition_CKPT_1\model.ckpt.
INFO:tensorflow:loss = 1.4932559, step = 0
INFO:tensorflow:Saving checkpoints for 41 into ./Considition_CKPT_1\mo